# Getting started with Quantinuum and OpenQASM 2.0 on Azure Quantum

This notebook shows how to send a basic quantum circuit expressed using the [OpenQASM 2.0 spec](https://github.com/Qiskit/openqasm/tree/OpenQASM2.x) to a Quantinuum target via the Azure Quantum service.

First, install `azure-quantum` and optionally `matplotlib` for plotting:

In [1]:
# To install the Azure Quantum client package, uncomment and run the line below:
#
# !pip install azure-quantum==0.19.2109.165653 --quiet
#
# We also recommend installing matplotlib, if you don't have it installed already:
# !pip install matplotlib --quiet

## Connecting to the Azure Quantum service

To connect to the Azure Quantum service, find the resource ID and location of your Workspace from the Azure portal here: https://portal.azure.com. Navigate to your Azure Quantum workspace and copy the values from the header.

<img src="../azure-quantum-resource-id.png" alt="How to retrieve the resource ID and location from an Azure Quantum workspace">

In [2]:
from azure.quantum import Workspace
from azure.quantum.target import Quantinuum

# Enter your workspace details here
# Find your resource ID and location via portal.azure.com
workspace = Workspace(
    resource_id="",
    location=""
)

Use `workspace.get_targets` to see what targets are currently available for the Quantinuum provider, including wait times. Running this method will trigger authentication to your Microsoft account, if you're not already logged in.

In [3]:
workspace.get_targets(provider_id="quantinuum")

[<Target name="quantinuum.qpu.h1-1", avg. queue time=0 s, Unavailable>,
 <Target name="quantinuum.sim.h1-1sc", avg. queue time=0 s, Available>,
 <Target name="quantinuum.qpu.h1-2", avg. queue time=236712 s, Unavailable>,
 <Target name="quantinuum.sim.h1-2sc", avg. queue time=0 s, Available>,
 <Target name="quantinuum.sim.h1-1e", avg. queue time=766 s, Available>]

### Submit a quantum circuit to the Quantinuum API validator

Note: The [Quantinuum API validator](https://docs.microsoft.com/azure/quantum/provider-quantinuum#api-validator) target will always return 0 on measurement.

Create a quantum circuit using the [OpenQASM 2.0 spec](https://github.com/Qiskit/openqasm/tree/OpenQASM2.x) representation. For example, the following example creates a Teleportation circuit:

In [4]:
# Create raw OpenQASM circuit. 
circuit = """OPENQASM 2.0;
include "qelib1.inc";
qreg q[3];
creg c0[1];
creg c1[3];
h q[0];
cx q[0], q[1];
x q[2];
h q[2];
cx q[2], q[0];
h q[2];
measure q[0] -> c1[0];
c0[0] = c1[0];
if (c0==1) x q[1];
c0[0] = 0;
measure q[2] -> c1[1];
c0[0] = c1[1];
if (c0==1) z q[1];
c0[0] = 0;
h q[1];
measure q[1] -> c1[2];
"""

To see if this circuit is valid, we can submit it to the Quantinuum API validator target. The following example uses the Quantinuum API validator, which returns a Job object. For more information, see [Azure Quantum Job](https://review.docs.microsoft.com/azure/quantum/optimization-job-reference).

In [5]:
target = workspace.get_targets(name="quantinuum.sim.h1-1sc")
job = target.submit(circuit)

Wait until the job is complete and then fetch the results.

In [6]:
results = job.get_results()
results

.......

{'c0': ['0'], 'c1': ['000']}

### Run on Quantinuum Simulator

It looks like the program was indeed valid! Now we can run the circuit and simulate the result with the Quantinuum simulator target:

In [7]:
target = Quantinuum(workspace=workspace, name="quantinuum.sim.h1-1e")
job = target.submit(circuit)

Wait until the job is complete and then fetch the results.

In [8]:
results = job.get_results()
results

...................
{'c0': ['0'], 'c1': ['111']}

